# Step 3 Target analysis of 670 and 700nm excitation TA data of WL of SCy6803

## Inspect data

In [ ]:
from cycler import cycler
from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.plot_traces import (
    plot_fitted_traces,
    select_plot_wavelengths,
)
from pyglotaran_extras.inspect import show_a_matrixes

In [ ]:
from pyglotaran_extras import plot_data_overview

DATA_PATH1 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_reva.ascii"
DATA_PATH2 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revb.ascii"
DATA_PATH3 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revc.ascii"
DATA_PATH4 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revd.ascii"
fig, axes = plot_data_overview(
    DATA_PATH4,
    nr_of_data_svd_vectors=3,
    linlog=False,
    linthresh=0.1,
    cmap="seismic",
    vmin=-7,
    vmax=7,
    use_svd_number=True,
)

In [ ]:
fig, axes = plot_data_overview(
    DATA_PATH4,
    nr_of_data_svd_vectors=4,
    linlog=False,
    linthresh=0.1,
    cmap="seismic",
    vmin=-7,
    vmax=7,
    use_svd_number=True,
    # cmap='bwr',datamin = -7,datamax=7
)
# TODO: the use of cmap, datamin, datamax requires an update to pyglotaran_extras

## Step 3a Target Analysis

### Used model and parameters

In [ ]:
target_model_path = "models/target_step1_Bulk_RP2_CA.yml"

In [ ]:
target_parameters_path = "models/target_step1_Bulk_RP2_CA.csv"
optimizedparameters = load_parameters(target_parameters_path)

#### Model file

In [ ]:
# Uncomment the following 2 lines to display the target model file in the notebook
# from glotaran.utils.ipython import display_file
# display_file(target_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition


#### Parameters file

In [ ]:
# Uncomment the next line and run the cell to print the starting values of the analysis
# These starting values have already been optimized, hence the name optimizedparameters

# optimizedparameters


## Create scheme 1 with Bulk to RP2 and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/target_step1_Bulk_RP2_CA.yml",
    parameters=load_parameters("models/target_step1_Bulk_RP2_CA.csv"),
    maximum_number_function_evaluations=9,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
target_result1 = optimize(target_scheme, raise_exception=True)

In [ ]:
# Just call the result to get the optimization result summary.
target_result1
# For easier copy-and-paste try:
# print(target_result)

### Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode

myFRLcolors = [
    "g",
    "b",
]

fig, axes = plt.subplots(1, 2, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 2)
plot_concentrations(
    target_result1.data["700TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
plot_concentrations(
    target_result1.data["700TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(
    target_result1.data["670TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_concentrations(
    target_result1.data["670TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_sas(target_result1.data["670TR2"], axes[1], cycler=custom_cycler)

axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
target_result1.optimized_parameters

### Residual analysis of all data

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_svd_of_residual(
    result_dataset, result_dataset2, result_dataset3, result_dataset4, linlog, linthresh
):
    fig, axes = plt.subplots(1, 2, figsize=(10, 2))
    custom_cycler = cycler(color=["tab:grey"])
    plot_lsv_residual(
        result_dataset,
        axes[0],
        indices=[0],
        linlog=linlog,
        linthresh=linthresh,
        cycler=custom_cycler,
    )
    plot_lsv_residual(
        result_dataset2, axes[0], indices=[0], linlog=linlog, linthresh=linthresh
    )
    custom_cycler = cycler(color=["tab:orange"])
    plot_lsv_residual(
        result_dataset3,
        axes[0],
        indices=[0],
        linlog=linlog,
        linthresh=linthresh,
        cycler=custom_cycler,
    )
    custom_cycler = cycler(color=["r"])
    plot_lsv_residual(
        result_dataset4,
        axes[0],
        indices=[0],
        linlog=linlog,
        linthresh=linthresh,
        cycler=custom_cycler,
    )
    axes[0].set_xlabel("Time (ps)")
    axes[0].get_legend().remove()
    axes[0].set_ylabel("")
    axes[0].set_title("residual 1st LSV")
    custom_cycler = cycler(color=["tab:grey"])
    plot_rsv_residual(result_dataset, axes[1], indices=[0], cycler=custom_cycler)
    plot_rsv_residual(result_dataset2, axes[1], indices=[0])
    custom_cycler = cycler(color=["tab:orange"])
    plot_rsv_residual(result_dataset3, axes[1], indices=[0], cycler=custom_cycler)
    custom_cycler = cycler(color=["r"])
    plot_rsv_residual(result_dataset4, axes[1], indices=[0], cycler=custom_cycler)
    axes[1].set_xlabel("Wavelength (nm)")
    axes[1].set_title("residual 1st RSV")
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")

    return fig, axes


fig, axes = plot_svd_of_residual(
    target_result1.data["670TR1"],
    target_result1.data["670TR2"],
    target_result1.data["700TR1"],
    target_result1.data["700TR2"],
    linlog=True,
    linthresh=10,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

## Step 3b Create scheme 2 introducing Red and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/target_step2_Bulk_RP2_CA_Red.yml",
    parameters=load_parameters("models/target_step2_Bulk_RP2_CA_Red.csv"),
    maximum_number_function_evaluations=9,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
target_result2 = optimize(target_scheme, raise_exception=True)

In [ ]:
# Just call the result to get the optimization result summary.
target_result2
# For easier copy-and-paste try:
# print(target_result2)

### Plot result for interpretation


In [ ]:
myFRLcolors = [
    "g",
    "r",
    "b",
]
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 3)
plot_concentrations(
    target_result2.data["700TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
plot_concentrations(
    target_result2.data["700TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(
    target_result2.data["670TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_concentrations(
    target_result2.data["670TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_sas(target_result2.data["670TR2"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
target_result2.optimized_parameters

### Residual analysis of all data

In [ ]:
fig, axes = plot_svd_of_residual(
    target_result2.data["670TR1"],
    target_result2.data["670TR2"],
    target_result2.data["700TR1"],
    target_result2.data["700TR2"],
    linlog=True,
    linthresh=10,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

## Step 3c Create scheme 3A introducing Ant1 and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/target_step3A_Bulk_RP2_CA_Red_Ant1.yml",
    parameters=load_parameters("models/target_step3A_Bulk_RP2_CA_Red_Ant1.csv"),
    maximum_number_function_evaluations=9,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
target_result3 = optimize(target_scheme, raise_exception=True)

In [ ]:
# Just call the result to get the optimization result summary.
target_result3
# For easier copy-and-paste try:
# print(target_result3)

### Plot result for interpretation


In [ ]:
myFRLcolors = [
    ColorCode.green,
    "g",
    "r",
    "b",
]
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 4)
plot_concentrations(
    target_result3.data["700TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
plot_concentrations(
    target_result3.data["700TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(
    target_result3.data["670TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_concentrations(
    target_result3.data["670TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_sas(target_result3.data["670TR2"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

## Step 3d Create scheme 3B introducing RP1 and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/target_step3B_Bulk_RP2_CA_Red_RP1.yml",
    parameters=load_parameters("models/target_step3B_Bulk_RP2_CA_Red_RP1.csv"),
    maximum_number_function_evaluations=9,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
target_result3 = optimize(target_scheme, raise_exception=True)

In [ ]:
# Just call the result to get the optimization result summary.
target_result3
# For easier copy-and-paste try:
# print(target_result3)

### Plot result for interpretation


In [ ]:
myFRLcolors = [
    "g",
    "r",
    ColorCode.cyan,
    "b",
]
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 4)
plot_concentrations(
    target_result3.data["700TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
plot_concentrations(
    target_result3.data["700TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(
    target_result3.data["670TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_concentrations(
    target_result3.data["670TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_sas(target_result3.data["670TR2"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

## Step 3e Create scheme 3C introducing both Ant1 and RP1 and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/target_step3C_Bulk_RP2_CA_Red_Ant1_RP1.yml",
    parameters=load_parameters("models/target_step3C_Bulk_RP2_CA_Red_Ant1_RP1.csv"),
    maximum_number_function_evaluations=9,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
        "RP1SADS": "guide/global670and700_670TR2_clp_br4.ascii",
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
target_result3 = optimize(target_scheme, raise_exception=True)

In [ ]:
# Just call the result to get the optimization result summary.
target_result3
# For easier copy-and-paste try:
# print(target_result)

### Plot result for interpretation


In [ ]:
myFRLcolors = [
    ColorCode.green,
    "g",
    "r",
    ColorCode.cyan,
    "b",
]
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 5)
plot_concentrations(
    target_result3.data["700TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
plot_concentrations(
    target_result3.data["700TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(
    target_result3.data["670TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_concentrations(
    target_result3.data["670TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_sas(target_result3.data["670TR2"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
target_result3.optimized_parameters

### Residual analysis of all data

In [ ]:
fig, axes = plot_svd_of_residual(
    target_result3.data["670TR1"],
    target_result3.data["670TR2"],
    target_result3.data["700TR1"],
    target_result3.data["700TR2"],
    linlog=True,
    linthresh=10,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

### Create the step 3C guidance data sets

In [ ]:
from glotaran.io import save_dataset
from glotaran.utils.io import create_clp_guide_dataset

for species in target_result3.data["670TR2"].species:
    clp_guide = create_clp_guide_dataset(target_result3.data["670TR2"], species.item())
    string_in_string = (
        "guide/target_step3C_Bulk_RP2_CA_Red_Ant1_RP1_clp_{}.ascii".format(
            species.item()
        )
    )
    save_dataset(clp_guide.data, string_in_string, allow_overwrite=True)

## Step 3f Create scheme 4 introducing RC and free Chla and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free.yml",
    parameters=load_parameters(
        "models/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free.csv"
    ),
    maximum_number_function_evaluations=9,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
        "Ant1SADS": "guide/target_step3C_Bulk_RP2_CA_Red_Ant1_RP1_clp_Ant1.ascii",
        "RP1SADS": "guide/target_step3C_Bulk_RP2_CA_Red_Ant1_RP1_clp_RP1.ascii",
        "freeSADS": "guide/global670and700_670TR2_clp_freeChla.ascii",
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
target_result4 = optimize(target_scheme, raise_exception=True)

In [ ]:
# Just call the result to get the optimization result summary.
target_result4
# For easier copy-and-paste try:
# print(target_result)

### Plot result for interpretation


In [ ]:
myFRLcolors = [ColorCode.green, "g", "r", "k", ColorCode.cyan, "b", "y"]
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 7)
plot_concentrations(
    target_result4.data["700TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
plot_concentrations(
    target_result4.data["700TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(
    target_result4.data["670TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_concentrations(
    target_result4.data["670TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_sas(target_result4.data["670TR2"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
# stop


In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
target_result4.optimized_parameters

### Residual analysis of all data

In [ ]:
fig, axes = plot_svd_of_residual(
    target_result4.data["670TR1"],
    target_result4.data["670TR2"],
    target_result4.data["700TR1"],
    target_result4.data["700TR2"],
    linlog=True,
    linthresh=1,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

There is no more structure in the 1st LSV of the residual matrices. However, the RMSE of the 670TR1 is 10% larger than the RMSE of the other three data sets.

### Residual analysis of the 670 nm excitation TR1 data

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual


def plot_residual_and_svd(result_dataset):
    fig, axes = plt.subplots(1, 3, figsize=(10, 2))
    plot_residual(result_dataset, axes[0])
    axes[0].get_legend().remove()
    axes[0].set_ylabel("Wavelength (nm)")
    axes[0].set_title("residual 670TR1")
    plot_lsv_residual(result_dataset, axes[1], indices=[0, 1])
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")
    axes[1].set_title("residual LSV1,2")
    plot_rsv_residual(result_dataset, axes[2], indices=[0, 1])
    axes[2].set_xlabel("Wavelength (nm)")
    axes[2].set_title("residual RSV1,2")
    axes[2].get_legend().remove()
    axes[2].set_ylabel("")

    return fig, axes


fig, axes = plot_residual_and_svd(target_result4.data["670TR1"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

LSV2 of the 670TR1 residual shows a trend on the timescale of 5 ps, and RSV2 is large around 700 nm. This can be solved by the splitting of the Red compartment in two parts. Therefore, we introduce a 2nd Red compartment. To smooth the SADS of the RC we will use a Spectral fit of the RC SADS estimated in step 4. To split the Red compartment we will guide the Red1 and Red2 SADS using the Spectral fit of the Red SADS from step 4 with two shifted guidance SADS.

### Create the step 4 SADS data sets

In [ ]:
# from glotaran.io import save_dataset
# from glotaran.utils.io import create_clp_guide_dataset

for species in target_result4.data["670TR2"].species:
    clp_guide = create_clp_guide_dataset(target_result4.data["670TR2"], species.item())
    string_in_string = (
        "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_{}.ascii".format(
            species.item()
        )
    )
    save_dataset(clp_guide.data, string_in_string, allow_overwrite=True)

### Spectral fit of the estimated SADS of the RC with a sum of two skewed gaussians

In [ ]:
# from pathlib import Path

# import matplotlib.pyplot as plt
from glotaran.io import load_dataset, load_model, load_parameters

# from glotaran.optimization.optimize import optimize
# from glotaran.project.scheme import Scheme
# from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle
# from pyglotaran_extras import plot_data_overview

plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

dataset = load_dataset(
    "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_RC.ascii"
)
# plot_data_overview(dataset)

In [ ]:
spectral_model = load_model("models/spectral_model.yml")
spectral_parameters = load_parameters("models/spectral_params_RC.yml")
spectral_model.validate(parameters=spectral_parameters)

spectral_scheme = Scheme(
    spectral_model,
    spectral_parameters,
    data={"dataset": dataset},
    maximum_number_function_evaluations=25,
)
spectral_result = optimize(spectral_scheme)
# spectral_result
# print(f"\n{'#'*3} Spectral Model - Optimization Result {'#'*3}\n")
# print(spectral_result)

# %%
# print(f"\n{'#'*3} Spectral Model - Optimized Parameters {'#'*3}\n")
spectral_result.optimized_parameters
# BUG in plot_overview
# plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
spectral_result

In [ ]:
from pyglotaran_extras.plotting.plot_overview import plot_overview

fig, axes = plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
spectral_result.optimized_parameters

In [ ]:
from glotaran.io import save_dataset

# string_in_string = "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_spectral_{}.ascii".format(species.item())
save_dataset(
    spectral_result.data["dataset"].fitted_data,
    "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_RC_fitted.ascii",
    allow_overwrite=True,
)

In [ ]:
spectral_resultRC = spectral_result

### Spectral fit of the Red SADS with two shifted species

In [ ]:
# from pathlib import Path

# import matplotlib.pyplot as plt
from glotaran.io import load_dataset, load_model, load_parameters

# from glotaran.optimization.optimize import optimize
# from glotaran.project.scheme import Scheme
# from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle
# from pyglotaran_extras import plot_data_overview

plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

dataset = load_dataset(
    "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_Red.ascii"
)
# plot_data_overview(dataset)

In [ ]:
spectral_model = load_model("models/spectral_model2splitRed.yml")
spectral_parameters = load_parameters("models/spectral_parameters2splitRed.csv")
spectral_model.validate(parameters=spectral_parameters)

spectral_scheme = Scheme(
    spectral_model,
    spectral_parameters,
    data={"dataset": dataset},
)
spectral_result = optimize(spectral_scheme)
# spectral_result
# print(f"\n{'#'*3} Spectral Model - Optimization Result {'#'*3}\n")
# print(spectral_result)

# %%
# print(f"\n{'#'*3} Spectral Model - Optimized Parameters {'#'*3}\n")
# spectral_result.optimized_parameters
# NB commented out because of BUG in spectral_result.optimized_parameters

In [ ]:
plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

### Spectral fits of the raw SADS of RC and Red

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
spectral_resultRC.data["dataset"].data.plot(ax=axes[0])
spectral_resultRC.data["dataset"].fitted_data.plot(ax=axes[0])
axes[0].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("SADS (mOD)")
axes[0].set_title("RC")
axes[0].axhline(0, color="k", linewidth=1)
spectral_result.data["dataset"].data.plot.line(x="spectral", ax=axes[1])
spectral_result.data["dataset"].fitted_data.plot.line(x="spectral", ax=axes[1])
spec1 = (
    spectral_result.data["dataset"].clp[:, 0]
    * spectral_result.data["dataset"].species_spectra[:, 0]
    + spectral_result.data["dataset"].clp[:, 1]
    * spectral_result.data["dataset"].species_spectra[:, 1]
)
spec1.plot.line(x="spectral", ax=axes[1])
spec2 = (
    spectral_result.data["dataset"].clp[:, 2]
    * spectral_result.data["dataset"].species_spectra[:, 2]
    + spectral_result.data["dataset"].clp[:, 3]
    * spectral_result.data["dataset"].species_spectra[:, 3]
)
spec2.plot.line(x="spectral", ax=axes[1])
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("Red")
axes[1].axhline(0, color="k", linewidth=1)
axes[1].get_legend().remove()

In [ ]:
from glotaran.io import save_dataset

# string_in_string = "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_spectral_{}.ascii".format(species.item())
save_dataset(
    spec1,
    "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_RC_spec1_fitted.ascii",
    allow_overwrite=True,
)
save_dataset(
    spec2,
    "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_RC_spec2_fitted.ascii",
    allow_overwrite=True,
)

The guidance spectra created here are used in the next step to guide the Red1 and Red2 SADS.

## Step 3g.i Create scheme 5 introducing Red2 and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2.yml",
    parameters=load_parameters(
        "models/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2.csv"
    ),
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
        "RCSADS": "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_RC_fitted.ascii",
        "Red1SADS": "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_RC_spec2_fitted.ascii",
        "Red2SADS": "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_RC_spec1_fitted.ascii",
        "Ant1SADS": "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_Ant1.ascii",
        "freeSADS": "guide/global670and700_670TR2_clp_freeChla.ascii",
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
target_result = optimize(target_scheme, raise_exception=True)

### Results and parameters

In [ ]:
# Just call the result to get the optimization result summary.
target_result
# For easier copy-and-paste try:
# print(target_result)

Note that this RMSE of is virtually the same as that reached in a global analysis with 5 and 4 lifetimes for the 670 and 700 nm excitation TA data, respectively.

In [ ]:
target_result5 = target_result

### Residual analysis of the 670 nm excitation TR1 data

In [ ]:
fig, axes = plot_residual_and_svd(target_result5.data["670TR1"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

The introduction of the 2nd Red compartment has led to a decrease in the RMSE and disappearance of the previous structure in LSV2 and RSV2 of 670TR1. Instead these are now dominated by the still imperfect description of the CA, cf. also the large residuals straddling time zero in (A) and (B).

### Plot result for interpretation


In [ ]:
myFRLcolors = [ColorCode.green, "g", "tab:orange", "r", "k", ColorCode.cyan, "b", "y"]

fig, axes = plt.subplots(1, 2, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 8)
plot_concentrations(
    target_result5.data["700TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
plot_concentrations(
    target_result5.data["700TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(
    target_result5.data["670TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_concentrations(
    target_result5.data["670TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler,
)
plot_sas(target_result5.data["670TR2"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

### Comparison of the estimated SADS (red) and the guidance spectra (black)
The guidance spectra are the SADS estimated with scheme 3C.
Alternatively, (smooth) shapes can be estimated with the help of spectral models or splines. 

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 7))
target_result.data["RCSADS"].data.plot(ax=axes[0, 0])
target_result.data["RCSADS"].fitted_data.plot(ax=axes[0, 0])
target_result.data["Red1SADS"].data.plot(ax=axes[0, 1])
target_result.data["Red1SADS"].fitted_data.plot(ax=axes[0, 1])
target_result.data["Red2SADS"].data.plot(ax=axes[0, 2])
target_result.data["Red2SADS"].fitted_data.plot(ax=axes[0, 2])
# target_result.data["RP1SADS"].data.plot(ax=axes[1,0])
# target_result.data["RP1SADS"].fitted_data.plot(ax=axes[1,0])
target_result.data["Ant1SADS"].data.plot(ax=axes[1, 1])
target_result.data["Ant1SADS"].fitted_data.plot(ax=axes[1, 1])
target_result.data["freeSADS"].data.plot(ax=axes[1, 2])
target_result.data["freeSADS"].fitted_data.plot(ax=axes[1, 2])
axes[0, 0].set_xlabel("")
axes[0, 0].set_ylabel("SADS (mOD)")
axes[0, 0].set_title("RC")
axes[0, 1].set_xlabel("")
axes[0, 1].set_ylabel("SADS (mOD)")
axes[0, 1].set_title("Red1")
axes[0, 2].set_xlabel("")
axes[0, 2].set_ylabel("SADS (mOD)")
axes[0, 2].set_title("Red2")
axes[1, 0].set_xlabel("Wavelength (nm)")
axes[1, 0].set_ylabel("SADS (mOD)")
axes[1, 0].set_title("RP1")
axes[1, 1].set_xlabel("Wavelength (nm)")
axes[1, 1].set_ylabel("SADS (mOD)")
axes[1, 1].set_title("Ant1")
axes[1, 2].set_xlabel("Wavelength (nm)")
axes[1, 2].set_ylabel("SADS (mOD)")
axes[1, 2].set_title("free")

In [ ]:
# stop

### Create the step 5 SADS data sets

In [ ]:
# from glotaran.io import save_dataset
# from glotaran.utils.io import create_clp_guide_dataset

for species in target_result5.data["670TR2"].species:
    clp_guide = create_clp_guide_dataset(target_result5.data["670TR2"], species.item())
    string_in_string = (
        "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_{}.ascii".format(
            species.item()
        )
    )
    save_dataset(clp_guide.data, string_in_string, allow_overwrite=True)

Now in order to compute smooth guidance SADS we fit the estimated SADS of RC, Red1 and Red2 with a sum of two skewed gaussians, and employ these fits as the new guidance SADS, employing heavier weights of 0.4 and 0.7 for Red1 and Red2.

### Fit of the estimated SADS of RC, Red1, Red2, and free with a sum of two skewed gaussians

In [ ]:
# from pathlib import Path

# import matplotlib.pyplot as plt
from glotaran.io import load_dataset, load_model, load_parameters

# from glotaran.optimization.optimize import optimize
# from glotaran.project.scheme import Scheme
# from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle
# from pyglotaran_extras import plot_data_overview

plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

dataset = load_dataset(
    "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_RC.ascii"
)
# plot_data_overview(dataset)

In [ ]:
spectral_model = load_model("models/spectral_model.yml")
spectral_parameters = load_parameters("models/spectral_params_RC.yml")
spectral_model.validate(parameters=spectral_parameters)

spectral_scheme = Scheme(
    spectral_model,
    spectral_parameters,
    data={"dataset": dataset},
    maximum_number_function_evaluations=25,
)
spectral_result = optimize(spectral_scheme)
# spectral_result
# print(f"\n{'#'*3} Spectral Model - Optimization Result {'#'*3}\n")
# print(spectral_result)

# %%
# print(f"\n{'#'*3} Spectral Model - Optimized Parameters {'#'*3}\n")
spectral_result.optimized_parameters
# BUG in plot_overview
# plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
spectral_result

In [ ]:
from pyglotaran_extras.plotting.plot_overview import plot_overview

fig, axes = plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
# stop

In [ ]:
from glotaran.io import save_dataset

# string_in_string = "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_spectral_{}.ascii".format(species.item())
save_dataset(
    spectral_result.data["dataset"].fitted_data,
    "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_RC_fitted.ascii",
    allow_overwrite=True,
)

In [ ]:
spectral_resultRC = spectral_result

In [ ]:
plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

dataset = load_dataset(
    "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_Red1.ascii"
)
# plot_data_overview(dataset)

In [ ]:
spectral_model = load_model("models/spectral_model.yml")
spectral_parameters = load_parameters("models/spectral_params.yml")
spectral_model.validate(parameters=spectral_parameters)

spectral_scheme = Scheme(
    spectral_model,
    spectral_parameters,
    data={"dataset": dataset},
)
spectral_result = optimize(spectral_scheme)
# spectral_result
# print(f"\n{'#'*3} Spectral Model - Optimization Result {'#'*3}\n")
# print(spectral_result)

# %%
# print(f"\n{'#'*3} Spectral Model - Optimized Parameters {'#'*3}\n")
spectral_result.optimized_parameters
# BUG in plot_overview
# plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
spectral_result

In [ ]:
from pyglotaran_extras.plotting.plot_overview import plot_overview

fig, axes = plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
from glotaran.io import save_dataset

# string_in_string = "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_spectral_{}.ascii".format(species.item())
save_dataset(
    spectral_result.data["dataset"].fitted_data,
    "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_Red1_fitted.ascii",
    allow_overwrite=True,
)

In [ ]:
spectral_resultRed1 = spectral_result

In [ ]:
plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

dataset = load_dataset(
    "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_Red2.ascii"
)
# plot_data_overview(dataset)

In [ ]:
spectral_model = load_model("models/spectral_model.yml")
spectral_parameters = load_parameters("models/spectral_params.yml")
spectral_model.validate(parameters=spectral_parameters)

spectral_scheme = Scheme(
    spectral_model,
    spectral_parameters,
    data={"dataset": dataset},
)
spectral_result = optimize(spectral_scheme)
# spectral_result
# print(f"\n{'#'*3} Spectral Model - Optimization Result {'#'*3}\n")
# print(spectral_result)

# %%
# print(f"\n{'#'*3} Spectral Model - Optimized Parameters {'#'*3}\n")
spectral_result.optimized_parameters
# BUG in plot_overview
# plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
spectral_result

In [ ]:
from pyglotaran_extras.plotting.plot_overview import plot_overview

fig, axes = plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
from glotaran.io import save_dataset

# string_in_string = "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_spectral_{}.ascii".format(species.item())
save_dataset(
    spectral_result.data["dataset"].fitted_data,
    "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_Red2_fitted.ascii",
    allow_overwrite=True,
)

In [ ]:
spectral_resultRed2 = spectral_result

In [ ]:
plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

dataset = load_dataset(
    "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_free.ascii"
)
# plot_data_overview(dataset)

In [ ]:
spectral_model = load_model("models/spectral_model.yml")
spectral_parameters = load_parameters("models/spectral_params_free.yml")
spectral_model.validate(parameters=spectral_parameters)

spectral_scheme = Scheme(
    spectral_model,
    spectral_parameters,
    data={"dataset": dataset},
    maximum_number_function_evaluations=25,
)
spectral_result = optimize(spectral_scheme)
# spectral_result
# print(f"\n{'#'*3} Spectral Model - Optimization Result {'#'*3}\n")
# print(spectral_result)

# %%
# print(f"\n{'#'*3} Spectral Model - Optimized Parameters {'#'*3}\n")
spectral_result.optimized_parameters
# BUG in plot_overview
# plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
spectral_result

In [ ]:
from pyglotaran_extras.plotting.plot_overview import plot_overview

fig, axes = plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
from glotaran.io import save_dataset

# string_in_string = "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_spectral_{}.ascii".format(species.item())
save_dataset(
    spectral_result.data["dataset"].fitted_data,
    "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_free_fitted.ascii",
    allow_overwrite=True,
)

### Spectral fits of the step5 SADS of RC, Red1, Red2

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
spectral_resultRC.data["dataset"].data.plot(ax=axes[0])
spectral_resultRC.data["dataset"].fitted_data.plot(ax=axes[0])
spectral_resultRed1.data["dataset"].data.plot(ax=axes[1])
spectral_resultRed1.data["dataset"].fitted_data.plot(ax=axes[1])
spectral_resultRed2.data["dataset"].data.plot(ax=axes[2])
spectral_resultRed2.data["dataset"].fitted_data.plot(ax=axes[2])
axes[0].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("SADS (mOD)")
axes[0].set_title("RC")
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("Red1")
axes[2].set_xlabel("Wavelength (nm)")
axes[2].set_ylabel("SADS (mOD)")
axes[2].set_title("Red2")

## Step 3g.ii Use scheme 5 with smooth guidance SADS and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2.yml",
    parameters=load_parameters(
        "models/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2.csv"
    ),
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH1,
        "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH3,
        "700TR2": DATA_PATH4,
        "RCSADS": "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_RC_fitted.ascii",
        "Red1SADS": "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_Red1_fitted.ascii",
        "Red2SADS": "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_Red2_fitted.ascii",
        "Ant1SADS": "guide/target_step4_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_clp_Ant1.ascii",
        "freeSADS": "guide/target_step5_Bulk_RP2_CA_Red_Ant1_RP1_RC_free_Red2_clp_free_fitted.ascii",
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
target_result = optimize(target_scheme, raise_exception=True)

### Results and parameters

In [ ]:
# Just call the result to get the optimization result summary.
target_result
# For easier copy-and-paste try:
# print(target_result)

To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about 52MB of disk space per save.

In [ ]:
save_result(
    result=target_result,
    result_path="results/target670and700/result.yaml",
    allow_overwrite=True,
)

### Plot result for interpretation


In [ ]:
myFRLcolors = [ColorCode.green, "g", "tab:orange", "r", "k", ColorCode.cyan, "b", "y"]
# myFRLcolors = [ ColorCode.green,"g","tab:orange",  "r", "k",ColorCode.cyan, "b", ColorCode.yellow]

fig, axes = plt.subplots(1, 2, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 8)
plot_concentrations(
    target_result.data["700TR1"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
plot_concentrations(
    target_result.data["700TR2"],
    axes[0],
    center_λ=0,
    linlog=True,
    cycler=custom_cycler2,
)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(
    target_result.data["670TR1"], axes[0], center_λ=0, linlog=True, cycler=custom_cycler
)
plot_concentrations(
    target_result.data["670TR2"], axes[0], center_λ=0, linlog=True, cycler=custom_cycler
)
plot_sas(target_result.data["670TR2"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("SADS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

### Residual analysis of all data

In [ ]:
fig, axes = plot_svd_of_residual(
    target_result.data["670TR1"],
    target_result.data["670TR2"],
    target_result.data["700TR1"],
    target_result.data["700TR2"],
    linlog=True,
    linthresh=1,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)

### Comparison of the estimated SADS (red) and the guidance spectra (black)
The guidance spectra are the SADS estimated with scheme 3C.
Alternatively, (smooth) shapes can be estimated with the help of spectral models or splines. 

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 7))
target_result.data["RCSADS"].data.plot(ax=axes[0, 0])
target_result.data["RCSADS"].fitted_data.plot(ax=axes[0, 0])
target_result.data["Red1SADS"].data.plot(ax=axes[0, 1])
target_result.data["Red1SADS"].fitted_data.plot(ax=axes[0, 1])
target_result.data["Red2SADS"].data.plot(ax=axes[0, 2])
target_result.data["Red2SADS"].fitted_data.plot(ax=axes[0, 2])
# target_result.data["RP1SADS"].data.plot(ax=axes[1,0])
# target_result.data["RP1SADS"].fitted_data.plot(ax=axes[1,0])
target_result.data["Ant1SADS"].data.plot(ax=axes[1, 1])
target_result.data["Ant1SADS"].fitted_data.plot(ax=axes[1, 1])
target_result.data["freeSADS"].data.plot(ax=axes[1, 2])
target_result.data["freeSADS"].fitted_data.plot(ax=axes[1, 2])
axes[0, 0].set_xlabel("")
axes[0, 0].set_ylabel("SADS (mOD)")
axes[0, 0].set_title("RC")
axes[0, 1].set_xlabel("")
axes[0, 1].set_ylabel("SADS (mOD)")
axes[0, 1].set_title("Red1")
axes[0, 2].set_xlabel("")
axes[0, 2].set_ylabel("SADS (mOD)")
axes[0, 2].set_title("Red2")
axes[1, 0].set_xlabel("Wavelength (nm)")
axes[1, 0].set_ylabel("SADS (mOD)")
axes[1, 0].set_title("RP1")
axes[1, 1].set_xlabel("Wavelength (nm)")
axes[1, 1].set_ylabel("SADS (mOD)")
axes[1, 1].set_title("Ant1")
axes[1, 2].set_xlabel("Wavelength (nm)")
axes[1, 2].set_ylabel("SADS (mOD)")
axes[1, 2].set_title("free")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
target_result.data["RCSADS"].data.plot(ax=axes[0])
target_result.data["RCSADS"].fitted_data.plot(ax=axes[0])
target_result.data["Red1SADS"].data.plot(ax=axes[1])
target_result.data["Red1SADS"].fitted_data.plot(ax=axes[1])
target_result.data["Red2SADS"].data.plot(ax=axes[2])
target_result.data["Red2SADS"].fitted_data.plot(ax=axes[2])
axes[0].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("SADS (mOD)")
axes[0].set_title("RC")
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SADS (mOD)")
axes[1].set_title("Red1")
axes[2].set_xlabel("Wavelength (nm)")
axes[2].set_ylabel("SADS (mOD)")
axes[2].set_title("Red2")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].axhline(0, color="k", linewidth=1)
axes[2].axhline(0, color="k", linewidth=1)

In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
target_result.optimized_parameters

In [ ]:
# stop

### Result plots

<sub>Note: The color scheme of the plots in this notebook may not match published figures.</sub>

### Fit quality

In [ ]:
target_result_TA = (
    target_result.data["670TR1"],
    target_result.data["670TR2"],
    target_result.data["700TR1"],
    target_result.data["700TR2"],
)
wavelengths = select_plot_wavelengths(target_result_TA, equidistant_wavelengths=True)
plot_fitted_traces(target_result_TA, wavelengths, linlog=True, linthresh=1)

The above command `plot_fitted_traces` is used to plot a selection of traces for a set of wavelengths (autogenerated using the `select_plot_wavelengths` function).
To show to make a manual selection of traces, and 'dress up the plot' see the code below, which reproduces Figure 2 of the iScience paper.

In [ ]:
# Reproduction of Figure 2 of the iScience paper
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        target_result_TA,
        [685, 700, 720, 760],
        linlog=True,
        linthresh=1,  # published figure uses 0.3 for easthetic reasons, but here 1 looks better
        axes_shape=(2, 2),
        figsize=(6, 4),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.orange,
                cc.red,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "670 nm excitation"
        elif i == 5:
            labels[i] = "700 nm excitation"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            line.set_linewidth(0.5)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()

### Overview 670 exc

In [ ]:
plot_overview(
    target_result.data["670TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=[
            ColorCode.green,
            "g",
            ColorCode.orange,
            "r",
            "k",
            ColorCode.cyan,
            "b",
            "y",
        ]
    ),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)

In [ ]:
plot_overview(
    target_result.data["670TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=[
            ColorCode.green,
            "g",
            ColorCode.orange,
            "r",
            "k",
            ColorCode.cyan,
            "b",
            "y",
        ]
    ),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)

### Overview 700 exc

In [ ]:
plot_overview(
    target_result.data["700TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=[
            ColorCode.green,
            "g",
            ColorCode.orange,
            "r",
            "k",
            ColorCode.cyan,
            "b",
            "y",
        ]
    ),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)

In [ ]:
# from pyglotaran_extras.plotting.style import PlotStyle

fig, axes = plot_overview(
    target_result.data["700TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=[
            ColorCode.green,
            "g",
            ColorCode.orange,
            "r",
            "k",
            ColorCode.cyan,
            "b",
            "y",
        ]
    ),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)

### Amplitude matrices

In [ ]:
show_a_matrixes(target_result)

### K-matrix of the target model

In [ ]:
compartments = target_scheme.model.initial_concentration["input700"].compartments

target_scheme.model.k_matrix["kmWL"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
).replace("To", "").replace("From", "&larr;").replace("rates", "k")